## Predicting Loan Repayment ##

## Key Takeaways: ##

**-Multiple Imputation with Chained Equations (MICE) can be used to cope with NAs in one's data set.**

**-A multi-factor logistic model outperformed both a simple and "smart" baseline methods and produced a respectable AUC value relative to competing models.**

**-A bivariate model using only lender-assigned interest rates as a predictor of non-payment performed poorly suggesting interest rates do not adequately capture risk of default.**

**-The model demonstrated some promise as the basis for a simple and risk-adjusted investment strategy, though it would need substantial adjustments to be placed in production.**

### The Problem ###

"In the lending industry, investors provide loans to borrowers in exchange for the promise of repayment with interest. If the borrower repays the loan, then the lender profits from the interest. However, if the borrower is unable to repay the loan, then the lender loses money. Therefore, lenders face the problem of predicting the risk of a borrower being unable to repay a loan.

To address this problem, we will use publicly available data from LendingClub.com, a website that connects borrowers and investors over the Internet. This dataset represents 9,578 3-year loans that were funded through the LendingClub.com platform between May 2007 and February 2010. The binary dependent variable not.fully.paid indicates that the loan was not paid back in full (the borrower either defaulted or the loan was "charged off," meaning the borrower was deemed unlikely to ever pay it back)."

(source: MITx)

### The Variables ###

To predict this dependent variable, we will use the following independent variables available to the investor when deciding whether to fund a loan:

**credit.policy**: 1 if the customer meets the credit underwriting criteria of LendingClub.com, and 0 otherwise.

**purpose**: The purpose of the loan (takes values "credit_card", "debt_consolidation", "educational", "major_purchase", "small_business", and "all_other").

**int.rate**: The interest rate of the loan, as a proportion (a rate of 11% would be stored as 0.11). Borrowers judged by LendingClub.com to be more risky are assigned higher interest rates.

**installment**: The monthly installments ($) owed by the borrower if the loan is funded.

**log.annual.inc**: The natural log of the self-reported annual income of the borrower.

**dti**: The debt-to-income ratio of the borrower (amount of debt divided by annual income).

**fico**: The FICO credit score of the borrower.

**days.with.cr.line**: The number of days the borrower has had a credit line.

**revol.bal**: The borrower's revolving balance (amount unpaid at the end of the credit card billing cycle).

**revol.util**: The borrower's revolving line utilization rate (the amount of the credit line used relative to total credit available).

**inq.last.6mths**: The borrower's number of inquiries by creditors in the last 6 months.

**delinq.2yrs**: The number of times the borrower had been 30+ days past due on a payment in the past 2 years.

**pub.rec**: The borrower's number of derogatory public records (bankruptcy filings, tax liens, or judgments).

(source: MITx)

### The Data ###

In [1]:
loans = read.csv("loans.csv")

In [2]:
str(loans)

'data.frame':	9578 obs. of  14 variables:
 $ credit.policy    : int  1 1 1 1 1 1 1 1 1 1 ...
 $ purpose          : Factor w/ 7 levels "all_other","credit_card",..: 3 2 3 3 2 2 3 1 5 3 ...
 $ int.rate         : num  0.119 0.107 0.136 0.101 0.143 ...
 $ installment      : num  829 228 367 162 103 ...
 $ log.annual.inc   : num  11.4 11.1 10.4 11.4 11.3 ...
 $ dti              : num  19.5 14.3 11.6 8.1 15 ...
 $ fico             : int  737 707 682 712 667 727 667 722 682 707 ...
 $ days.with.cr.line: num  5640 2760 4710 2700 4066 ...
 $ revol.bal        : int  28854 33623 3511 33667 4740 50807 3839 24220 69909 5630 ...
 $ revol.util       : num  52.1 76.7 25.6 73.2 39.5 51 76.8 68.6 51.1 23 ...
 $ inq.last.6mths   : int  0 0 1 1 0 0 0 0 1 1 ...
 $ delinq.2yrs      : int  0 0 0 0 1 0 0 0 0 0 ...
 $ pub.rec          : int  0 0 0 0 0 0 1 0 0 0 ...
 $ not.fully.paid   : int  0 0 0 0 0 0 1 1 0 0 ...


In [3]:
summary(loans)

 credit.policy                 purpose        int.rate       installment    
 Min.   :0.000   all_other         :2331   Min.   :0.0600   Min.   : 15.67  
 1st Qu.:1.000   credit_card       :1262   1st Qu.:0.1039   1st Qu.:163.77  
 Median :1.000   debt_consolidation:3957   Median :0.1221   Median :268.95  
 Mean   :0.805   educational       : 343   Mean   :0.1226   Mean   :319.09  
 3rd Qu.:1.000   home_improvement  : 629   3rd Qu.:0.1407   3rd Qu.:432.76  
 Max.   :1.000   major_purchase    : 437   Max.   :0.2164   Max.   :940.14  
                 small_business    : 619                                    
 log.annual.inc        dti              fico       days.with.cr.line
 Min.   : 7.548   Min.   : 0.000   Min.   :612.0   Min.   :  179    
 1st Qu.:10.558   1st Qu.: 7.213   1st Qu.:682.0   1st Qu.: 2820    
 Median :10.928   Median :12.665   Median :707.0   Median : 4140    
 Mean   :10.932   Mean   :12.607   Mean   :710.8   Mean   : 4562    
 3rd Qu.:11.290   3rd Qu.:17.950   3rd 

In [9]:
unpaid = round((sum(loans$not.fully.paid)/nrow(loans))*100,2)
cat(paste(unpaid,"% of loans are not fully paid."))

16.01 % of loans are not fully paid.

Columns missing at least one value are listed below. We'll be filling these in, as we would like to predict the risk for all borrowers and not simply the ones for whom we have a complete record.

In [23]:
# https://stackoverflow.com/questions/20364450
na_values = colnames(loans)[apply(is.na(loans), 2, any)]
na_values

[1] "log.annual.inc"    "days.with.cr.line" "revol.util"       
[4] "inq.last.6mths"    "delinq.2yrs"       "pub.rec"

We will use multiple imputation by chained equations (a/k/a "MICE") to computationally fill in missing values using data available to us.

In [38]:
install.packages("mice")

Warning message in install.packages("mice"):
“installation of package ‘mice’ had non-zero exit status”Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [39]:
library(mice)

Warning message:
“package ‘mice’ was built under R version 3.6.1”Loading required package: lattice

Attaching package: ‘mice’

The following objects are masked from ‘package:base’:

    cbind, rbind



In [40]:
set.seed(144)

# set vars.for.imputation to all variables in the data frame except for not.fully.paid, 
# to impute the values using all of the other independent variables.
vars.for.imputation = setdiff(names(loans), "not.fully.paid")

imputed = complete(mice(loans[vars.for.imputation]))

loans[vars.for.imputation] = imputed


 iter imp variable
  1   1  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  1   2  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  1   3  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  1   4  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  1   5  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  2   1  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  2   2  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  2   3  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  2   4  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  2   5  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  3   1  log.annual.inc  days.with.cr.line  revol.

## The Model ##

We start by splitting our data set into training and testing portions and building a logistic model using all independent variables.

In [42]:
install.packages("caTools")
library(caTools)

Warning message in install.packages("caTools"):
“installation of package ‘caTools’ had non-zero exit status”Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [44]:
set.seed(144)

spl = sample.split(loans$not.fully.paid, .70)
train = subset(loans, spl==TRUE)
test = subset(loans, spl==FALSE)

In [47]:
logit1 = glm(not.fully.paid ~ ., data=train, family=binomial)

In [48]:
summary(logit1)


Call:
glm(formula = not.fully.paid ~ ., family = binomial, data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2008  -0.6213  -0.4953  -0.3609   2.6389  

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)                9.250e+00  1.552e+00   5.959 2.54e-09 ***
credit.policy             -3.417e-01  1.009e-01  -3.388 0.000704 ***
purposecredit_card        -6.124e-01  1.344e-01  -4.557 5.18e-06 ***
purposedebt_consolidation -3.199e-01  9.179e-02  -3.485 0.000493 ***
purposeeducational         1.351e-01  1.753e-01   0.771 0.440814    
purposehome_improvement    1.728e-01  1.479e-01   1.168 0.242901    
purposemajor_purchase     -4.828e-01  2.008e-01  -2.404 0.016215 *  
purposesmall_business      4.123e-01  1.418e-01   2.907 0.003653 ** 
int.rate                   6.434e-01  2.085e+00   0.309 0.757592    
installment                1.274e-03  2.092e-04   6.091 1.12e-09 ***
log.annual.inc            -4.328e-01  

As the output above indicates, 11 of our independent variables are significant at the .01 and .001 levels.

Assume we have two loan applicants, Alice and Bob. Alice has a FICO score of 700, while Bob has a FICO score of 710. Their applications are the same in all respects. Using our model, we can preict the differential in log odds between the two applicants as follows.

In [51]:
-9.408e-03*(700-710)

[1] 0.09408

The odds of Alice's loan not being paid back should be higher than that of Bob's, but by how much?

In [52]:
exp(0.09408)

[1] 1.098648

Alice is approximately 1.099 times more likely to default than Bob. 

### Out-Sample-Testing ###

In [53]:
logit_pred = predict(logit1, newdata=test, type="response")

In [63]:
test$pred_risk = logit_pred

In [94]:
confusion_mtx = function(df, observed_y, predicted_y, threshold) {
    
    # Input: dataframe, response variable name as string, vector of 
    # predicted values as variable, threshold value as float
    # 
    # Output: Confusion matrix to assess model performance
    
    cmtx=table(df[[observed_y]], predicted_y > threshold)
    return (cmtx)
}

In [99]:
cmtx=confusion_mtx(test, "not.fully.paid", logit_pred, .5)
addmargins(cmtx)

,FALSE,TRUE,Sum
0,2400,13,2413
1,457,3,460
Sum,2857,16,2873


In [98]:
confusion_accuracy = function(confusion_matrix) {
    
    # Input: confusion matrix
    #
    # Output: Message stating the accuracy of the model based on the data
    # contained in the matrix.
    
    accuracy = sum(diag(prop.table(confusion_matrix)))

    return (cat(paste("Based on the confusion matrix, the out-of-sample accuracy of our model is", round(accuracy,4)*100, "%.")))
}

confusion_accuracy(cmtx)

Based on the confusion matrix, the out-of-sample accuracy of our model is 83.64 %.

As per usual, we now ask how our model compares to a baseline prediction derived from the test set.

In [127]:
cat(paste("Our baseline model has an accuracy of ", round(((cmtx[[1]])/nrow(test)), 4)*100, "%."))

Our baseline model has an accuracy of  83.54 %.

The difference between our model's accuracy and that of the baseline is hardly overwhelming (83.6% vs 83.5%). This begs the question whether or not its still usable. 

### The ROC Curve ###

We'll consult the Receiver Operator Characteristic (ROC) to assess model performance further. 

In [116]:
install.packages("ROCR")
library(ROCR)

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Loading required package: gplots
Warning message:
“package ‘gplots’ was built under R version 3.6.1”
Attaching package: ‘gplots’

The following object is masked from ‘package:stats’:

    lowess



Below, we calculate the area under the curve (AUC) for the test set using the .5 threshold. It seems that our model does better than the AUC benchmark of .5, indicative of a random process akin to a coin toss. 

In [121]:
pred = prediction(test$pred_risk, test$not.fully.paid)
cat(paste(round(as.numeric(performance(pred, "auc")@y.values), 4)*100),"%")

67.21 %

### An Alternative Model ###

Interest rates capture the risk associated with repayment. Though it was not statistically significant in the first model, this was likely due to multicollinearity. Here, we construct a bivariate model using only interest rate that will serve as a new **smart baseline**.

In [122]:
logit2 = glm(not.fully.paid ~ int.rate, data=train, family=binomial)

In [123]:
logit2_pred = predict(logit2, newdata=test, type="response")

Interestingly, the summary data below indicate the maximum probabiity of default is 42.66%. Using a 50% threshold, this implies absolutely no loans will be flagged as "not.fully.paid." This is every investment manager's fantasy portfolio -- a collection of loans with no defaults. It is a scenario, however, that seems highly unlikely. 

In [211]:
summary(logit2_pred)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.06196 0.11549 0.15077 0.15963 0.18928 0.42662 

Below is the AUC calculation for our second model. 

In [143]:
pred2 = prediction(logit2_pred, test$not.fully.paid)
cat(paste(round(as.numeric(performance(pred2, "auc")@y.values), 4)*100),"%")

62.39 %

The AUC above is worse than that of our first model. This result is actually quite significant. In the words of one hedge fund manager, assigning an interest rate to a corporate loan, "Is not a scientific process." Given that a model using solely the interest rate a lender assigns to a credit as a predictor of defaault undrperformed a more nuanced model that could still be improved, we can say that, indeed, interest rate assignment is **not** a scientific process. Perhaps it should be, as it is interpreted as the sine qua non of default expectations and a figure that fully compensates the lender in the event of default. Perhaps this is not the case. 

### A Simplified Investment Scenario ###

Assuming interest compunds continuously for every investment, $I$, according to the formula $I\cdot\text{e}^{rt}$, has a three-year investment term, and total loss if the loan is not paid in full, we calculate the maximum expected profit of a $10 investment in any loan in our test set.

In [145]:
test$profit = exp(test$int.rate*3) - 1

test$profit[test$not.fully.paid == 1] = -1

In [153]:
cat(paste("The max profit for any loan in our test set under the assumptions outlined above is $", round(max(test$profit)*10,2),"."))

The max profit for any loan in our test set under the assumptions outlined above is $ 8.89 .

### A Risk Based Investment Approach ###

We wish to invest in the highest yielding loans that have the lowest risk of default. In this section, we do just that.

In [154]:
high_yield = subset(test, test$int.rate >.15)

In [183]:
cat(paste("The mean profit of a $1 investment in a high-yield loan (int. rate > 15%) in our test set is", round(mean(high_yield$profit),4),"."))

The mean profit of a $1 investment in a high-yield loan (int. rate > 15%) in our test set is 0.2251 .

In [173]:
cat(paste("The percentage of high-yield loans (int. rate > 15%) not fully repaid in our test is", round(prop.table(table(high_yield$not.fully.paid))[[2]]*100,2),"%."))

The percentage of high-yield loans (int. rate > 15%) not fully repaid in our test is 25.17 %.

We will further refine our high-yield portfolio by setting a cutoff for the maximum predicted risk for loans we will in this cohort. Additionally, we'll limit the size of our portfolio to 100 investments. 

In [199]:
max_risk = round(sort(high_yield$pred_risk, decreasing=FALSE)[100],4)*100
max_risk

[1] 17.71

In [182]:
high_yield2 = subset(high_yield, pred_risk <= max_risk)
nrow(high_yield2)

[1] 100

In [205]:
cat(paste("The profit from a $1 investment in every loan in our refined high-yield portfolio is         $", round(sum(high_yield2$profit),2),"."))

The profit from a $1 investment in every loan in our refined high-yield portfolio is         $ 31.28 .

In [212]:
cat(paste("The percentage of high-yield loans in our refined portfolio not fully repaid in our test is ", round(prop.table(table(high_yield2$not.fully.paid))[[2]]*100,2),"%."))

The percentage of high-yield loans in our refined portfolio not fully repaid in our test is  19 %.

**In summary, we used our first model to create a high-yield investment portfolio that maximized our expected profit while minimizing our exposure to loans most likely to default.**